# PC2 - Grupo 2 (Julia)

#### Integrantes
- GARCIA RODRIGUEZ, EMILIO ALONSO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SALAMANCA FERNANDEZ, LUCAS PABLO
- SILVA ANDUJAR, NICOLAS

## 1. Loading and processing the data

In [2]:
using Pkg
Pkg.add("Distributions")
Pkg.add("GLMNet")
Pkg.add("HTTP")
Pkg.add("StatsModels")
using CSV
using Distributions
using DataFrames
using Dates
using Plots
using Random
using LinearAlgebra
using LaTeXStrings

# using MLBase
using Statistics
using GLMNet


   Resolving package versions...
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Manifest.toml`


1. Load de data

In [3]:
using CSV
using DataFrames
using HTTP

data = CSV.read(
        download("https://raw.githubusercontent.com/d2cml-ai/CausalAI-Course/main/data/wage2015_subsample_inference.csv"), 
        DataFrame, 
        types = Dict(:occ2 => String, :ind2 => String)
);
select!(data, Not("rownames"))



Row,wage,lwage,sex,shs,hsg,scl,clg,ad,mw,so,we,ne,exp1,exp2,exp3,exp4,occ,occ2,ind,ind2
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String,Float64,String
1,9.61538,2.26336,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,7.0,0.49,0.343,0.2401,3600.0,11,8370.0,18
2,48.0769,3.8728,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,3050.0,10,5070.0,9
3,11.0577,2.40313,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,18.0,3.24,5.832,10.4976,6260.0,19,770.0,4
4,13.9423,2.63493,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,25.0,6.25,15.625,39.0625,420.0,1,6990.0,12
5,28.8462,3.36198,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,22.0,4.84,10.648,23.4256,2015.0,6,9470.0,22
6,11.7308,2.46222,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.01,0.001,0.0001,1650.0,5,7460.0,14
7,19.2308,2.95651,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,42.0,17.64,74.088,311.17,5120.0,17,7280.0,14
8,19.2308,2.95651,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,37.0,13.69,50.653,187.416,5240.0,17,5680.0,9
9,12.0,2.48491,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,31.0,9.61,29.791,92.3521,4040.0,13,8590.0,19


2. Generate de Extra-Flexible Model. This model contains all two-way interactions between the experience polynomials and the indicator variables. 

In [4]:
using StatsModels 
# Extra-flexible model
eflex_formula = @formula(lwage ~ 1 + sex + (exp1 + exp2 + exp3 + exp4) * (hsg + scl + clg + ad + so + we + ne + occ2 + ind2) + (hsg + scl + clg + ad) * (so + we + ne + occ2 + ind2) + (so + we + ne) * (occ2 + ind2) + occ2 * ind2)
X_eflex = modelmatrix(eflex_formula, data);

println(size(X_eflex))

println(any(ismissing, X_eflex))

(5150, 965)
false


In [5]:
for term in terms(eflex_formula)
    println(term)
end

lwage
1
sex
exp1
exp2
exp3
exp4
hsg
scl
clg
ad
so
we
ne
occ2
ind2


2.1 Generate the array for the outcome variable $Y$ and normalize it

In [6]:
y = data[:, "lwage"]
y_mean = mean(y)
y_std = std(y)

y_norm = (y .- y_mean) ./ y_std


5150-element Vector{Float64}:
 -1.2402545501637507
  1.581415943901389
 -0.9952235701262617
 -0.5888284120346895
  0.6858352078834439
 -0.8916287370734166
 -0.02502721376252582
 -0.02502721376252582
 -0.8518460993196265
 -0.02502721376252582
  ⋮
  0.25770849799681306
 -0.2591347787012701
  0.3661886075001642
 -0.6009613493971904
 -0.48781225590462607
  0.29461938662075393
  1.1902001226386991
  0.9199427728221882
 -0.20974552813450043

2.2 Generate the array for the predictors $X$ (without intercept) and normalize its columns

In [7]:
X = X_eflex

exp_indices = [3, 4, 5, 6]  # Asegúrate de que estos índices son correctos según tus datos

for col in exp_indices
    col_mean = mean(X[:, col])
    col_std = std(X[:, col])
    
    if col_std != 0
        X[:, col] = (X[:, col] .- col_mean) ./ col_std
    else
        X[:, col] .= 0.0  # Si la desviación estándar es cero, puedes asignar 0 a toda la columna
    end
end


3. Split between training and testing sample. The testing sample should be 10% of the total

In [8]:
train_sample = rand(Float64, size(data)[1]) .< 0.9
test_sample = .!(train_sample)
y_train, y_test = y[train_sample], y[test_sample]
X_train, X_test = X[train_sample, :], X[test_sample, :];

# 2. Creating the Lasso Cross-Validation Procedure

4. Program a function that generates a logarithmically spaced grid. The input arguments should be the natural logarithms of the lower and upper bounds of the grid, as well as the natural logarithm of the spacing between each element of the grid. The output should be the logarithmically spaced grid, meaning that if we take the natural logarithm of each entry in the grid, they will be equally spaced. This will be the grid of values for $λ$ values to try during cross-validation.

In [9]:
function log_space_grid(log_lower_bound::Float64, log_upper_bound::Float64, log_spacing::Float64)
    grid = exp.(log_lower_bound:log_spacing:log_upper_bound)
    return grid
end

# Test the function 
log_lower_bound = -3.0  # log(0.001)
log_upper_bound = 3.0   # log(20.085)
log_spacing = 1.0       # Log step size

# Call the function and store the result
grid = log_space_grid(log_lower_bound, log_upper_bound, log_spacing)

# Print the result to verify
println("Generated log-space grid: ", grid)

Generated log-space grid: [0.049787068367863944, 0.1353352832366127, 0.36787944117144233, 1.0, 2.718281828459045, 7.38905609893065, 20.085536923187668]


5. Program a function to generate $k$ folds. It should take as input the array to be split rowwise and the number of folds desired. It should output a list of $k$ 1d arrays of booleans; these arrays should all be the same length as the number of rows in the input array, and when they are all summed together they should add up to an array of all true values. Create your own procedure for splitting. You can aid yourself with third party packages like numpy in Python or Statistics in Julia, but do not use a pre-programmed third party splitting procedure like sk-learns's KFolds in Python.

In [10]:
function gen_kfolds(X::AbstractArray, k::Int)
    n = size(X,1)

    indices = shuffle(1:n)

    folds = []
    fold_size = Int(floor(n / k))
    remainder = n % k

    start = 1
    for i in 1:k
        extra = i <= remainder ? 1 : 0  # Some folds will have 1 extra row if remainder > 0
        # Ensure we don't go out of bounds
        stop = min(start + fold_size + extra - 1, n)  # Make sure 'stop' is within bounds
        fold_indices = indices[start:stop]
        start += fold_size + extra

        # Create a boolean array to mark which rows belong to the current fold
        fold = falses(n)
        fold[fold_indices] .= true
        push!(folds, fold)
    end

    return folds
end

#Test function 
# Create a sample dataset
X_sample = rand(10, 5)  # A random dataset with 10 rows and 5 columns

# Test the gen_kfolds function with k=3 (3 folds)
k = 3
folds = gen_kfolds(X_sample, k)

# Print the result to verify
for (i, fold) in enumerate(folds)
    println("Fold $i:")
    println(fold)
    println("Training set size: ", sum(.!fold))  
    println("Test set size: ", sum(fold))        
    println()
end

total_test_set_size = sum([sum(fold) for fold in folds])
println("Total test set size across all folds: ", total_test_set_size)

Fold 1:
Bool[1, 0, 0, 0, 1, 0, 1, 0, 1, 0]
Training set size: 6
Test set size: 4

Fold 2:
Bool[0, 0, 0, 1, 0, 1, 0, 1, 0, 0]
Training set size: 7
Test set size: 3

Fold 3:
Bool[0, 1, 1, 0, 0, 0, 0, 0, 0, 1]
Training set size: 7
Test set size: 3

Total test set size across all folds: 10


6. Program a function that integrates those that you programmed in the last two items to find the value of λ that minimizes the testing mean square error across folds. It should take the following inputs:

- Y: an array for the outcome variable.
- X: an array of predictors.
- lambda_bounds: the lower and upper bounds of the grid of lambda values.
- k: number of folds...



In [18]:
using GLMNet
# If Lasso.jl is being used too, it's best to avoid conflicts by specifying the package when calling functions

function lasso_cv(X::AbstractArray, y::AbstractArray, lambda_bounds::Tuple{Float64, Float64}, k::Int)
    # Generate a log-spaced grid of lambdas
    log_lower_bound, log_upper_bound = lambda_bounds
    lambdas = log_space_grid(log_lower_bound, log_upper_bound, 0.1)  

    folds = gen_kfolds(X, k)
    
    n_lambdas = length(lambdas)
    mse_per_lambda = zeros(n_lambdas)
    
    for (j, lamda) in enumerate(lambdas)
        mse_fold = 0.0  
        
        for fold in folds
            X_train = X[.!fold, :]
            y_train = y[.!fold]
            X_test = X[fold, :]
            y_test = y[fold]
            
            # Lasso model
            model = glmnet(X_train, y_train, alpha=1.0, lambda=[lamda]) 

            # Make predictions on the test set
            y_pred = GLMNet.predict(model, X_test)
            
            # Calculate MSE
            mse_fold += mean((y_pred .- y_test).^2)
        end
        
        # Store average MSE for the current lambda across all folds
        mse_per_lambda[j] = mse_fold / k
    end

    # Find optimal lambda with minimum MSE
    optimal_lambda_idx = argmin(mse_per_lambda)
    optimal_lambda = lambdas[optimal_lambda_idx]

    # Fit the optimal model on the full dataset
    optimal_model = glmnet(X, y, alpha=1.0, lambda=[optimal_lambda])

    # Extract the coefficients using the GLMNet's `coef` function explicitly
    optimal_coefs = optimal_model.betas

    # Return dictionary with results
    return Dict(
        "optimal_lambda" => optimal_lambda,
        "optimal_coefs" => optimal_coefs,  # Coefficients from the optimal model
        "all_lambdas" => lambdas,
        "all_mse" => mse_per_lambda
    )
end


lasso_cv (generic function with 1 method)

7. Program a function for predicting the outcome variable through model estimated with the optimal lambda. It should take as inputs

- optimal_model: A dictionary with the values outputed by the function defined for the previous point.
- X: an array of predictors.
- The output should be an array of predicted values.

In [39]:
function predict_outcome(optimal_model::Dict, X::AbstractArray)
    
    optimal_coefs = optimal_model["optimal_coefs"]
    
    y_pred = X * optimal_coefs
    
    return y_pred
end


predict_outcome (generic function with 2 methods)

# 3.Applying the Lasso Cross-Validation Procedure

8. Fit a simple OLS model with the training sample.

In [34]:
# Add a column of ones to X_train to account for the intercept
X_train_with_intercept = hcat(ones(size(X_train, 1)), X_train)

# Fit the OLS model
ols = lm(X_train_with_intercept, y_train)

LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}:

Coefficients:
──────────────────────────────────────────────────────────────────────────────
              Coef.   Std. Error       t  Pr(>|t|)     Lower 95%     Upper 95%
──────────────────────────────────────────────────────────────────────────────
x1      3.33459        0.467874     7.13    <1e-11    2.41728       4.25189
x2      0.0          NaN          NaN       NaN     NaN           NaN
x3     -0.0682883      0.0170463   -4.01    <1e-04   -0.101709     -0.0348676
x4     -1.61246        3.07599     -0.52    0.6002   -7.6432        4.41828
x5      8.08116        8.68816      0.93    0.3524   -8.95272      25.115
x6    -11.6878         9.73703     -1.20    0.2301  -30.7781        7.40244
x7      5.4273         3.89806      1.39    0.1639   -2.21517      13.0698
x8     -0.847937       1.30549     -0.65    0.5160   -3.40745       1.71158
x9     -0.575467   

9. Find the optimal lambda and its related coefficients with the function programmed in 3. and the training sample. The natural logarithm of the numbers in the grid of $\lambda$s should be spaced by 0.1, and the natural logarithm of the lower and upper bounds of the grid should be -7 and 7 respectively. For the k-folds procedure, use 5 folds. Print the lambda and the coefficients.

In [14]:
lambda_bounds = (exp(-7.0), exp(7.0)) 
k_folds = 5  
lasso_man = lasso_cv(X_train, y_train, lambda_bounds, k_folds)

Dict{String, Any} with 4 entries:
  "all_mse"        => [0.322608, 0.322608, 0.322608, 0.322608, 0.322608, 0.3226…
  "optimal_lambda" => 1.00091
  "optimal_coefs"  => 965x1 CompressedPredictorMatrix:…
  "all_lambdas"    => [1.00091, 1.10618, 1.22252, 1.35109, 1.49319, 1.65023, 1.…

10. Now use each language's hdm package to fit a Lasso model with the theoretical optimal lambda value. You can find the documentation of the package for each language in the following links

In [35]:
using Pkg
Pkg.add("HDMjl")

using HDMjl
hdm_reg = rlasso(X_train, y_train, post = false)


   Resolving package versions...
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Nicolas Silva\.julia\environments\v1.10\Manifest.toml`


Dict{String, Any} with 15 entries:
  "tss"          => 1480.83
  "dev"          => [-0.709269, -0.569507, -0.337705, 0.389344, -0.0161215, -0.…
  "model"        => [1.0 1.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0; … ; 1.0 0.0 … 0.0 0.…
  "loadings"     => [0.0, 0.248149, 0.495302, 0.495897, 0.496333, 0.495936, 0.2…
  "sigma"        => 0.50256
  "lambda0"      => 651.913
  "lambda"       => [0.0, 161.772, 322.894, 323.282, 323.566, 323.307, 132.415,…
  "intercept"    => 2.92775
  "iter"         => 8
  "residuals"    => [-0.689413, -0.455123, -0.754513, 0.263297, 0.0759996, 0.19…
  "rss"          => 1159.79
  "index"        => Bool[0, 0, 1, 0, 0, 0, 1, 0, 1, 1  …  0, 0, 0, 0, 0, 0, 0, …
  "beta"         => [0.0, 0.0, 0.037477, 0.0, 0.0, 0.0, -0.0844814, 0.0, 0.1689…
  "options"      => Dict{String, Any}("intercept"=>true, "post"=>false, "meanx"…
  "coefficients" => [2.92775, 0.0, 0.0, 0.037477, 0.0, 0.0, 0.0, -0.0844814, 0.…

11. Report the testing MSE and R2 for the OLS model and, the cross-validation lambda Lasso model, and the hdm theoretical lambda model. Use the function programmed in 4. to compute the predictions of the cross-validation Lasso model.

In [50]:
#OLS
ols_mse = mean(residuals(ols) .^ 2)
ols_r2 = 1 - ols_mse / var(y_train)
println("OLS Model: ")
println("MSE: ", ols_mse)
println("R^2: ", ols_r2)

# CV-Lasso: Ajustar y predecir con Lasso
optimal_model = glmnet(X_train, y_train, alpha=1.0, lambda=[optimal_lambda])

# Predicciones en el conjunto de entrenamiento
y_pred_train = GLMNet.predict(optimal_model, X_train)
lasso_residuals = y_train .- y_pred_train

# Cálculo de MSE y R^2 para el conjunto de entrenamiento
lasso_mse = mean(lasso_residuals.^2)
println("Lasso Training MSE: ", lasso_mse)

r2 = 1 - lasso_mse / var(y_train)
println("Lasso Training R^2: ", r2)



OLS Model: 
MSE: 0.17529867525210394
R^2: 0.4564052441489479
Lasso Training MSE: 0.3224102272484502
Lasso Training R^2: 0.0002177226213805339


In [48]:
# HDMjl-Lasso
HDM_mse = mean(hdm_reg["residuals"] .^ 2)
HDM_r2 = 1 - HDM_mse / var(y_train)

println("HDM Lasso Model: ")
println("MSE: ", HDM_mse)
println("R^2: ", HDM_r2)

HDM Lasso Model: 
MSE: 0.2525114648192755
R^2: 0.21697121857526325
